In [71]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random

from scipy import sparse
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

import plotnine
from plotnine import *

import pickle

In [72]:
# 경로 설정
data_path = './data'
saved_path = './code/saved'
output_path = './code/submission'

In [73]:
# 데이터 불러오기 
history_df = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')
search_df = pd.read_csv(os.path.join(data_path, 'search_data.csv'), encoding='utf-8')
profile_df = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')
meta_df = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')
we_df = pd.read_csv(os.path.join(data_path, 'watch_e_data.csv'), encoding='utf-8')
buy_df = pd.read_csv(os.path.join(data_path, 'buy_data.csv'), encoding='utf-8')

In [74]:
meta_plus_df = pd.read_csv(os.path.join(data_path, 'meta_data_plus.csv'), encoding='utf-8')

In [75]:
meta_plus_df

,album_id,keyword_type,keyword_name,keyword_value
0,749,05010101,시리즈,0
1,749,05010302,EBS,0
2,749,05010401,3세,0
3,749,05010402,4세,0
4,749,05010407,1세,0
...,...,...,...,...
767943,11629,05040104,신나는,0
767944,11629,0504010b,따라하고 싶은,0
767945,11629,05070e01,인체,0
767946,11629,05080401,캐릭터 친구들,0


In [76]:
tag = meta_plus_df.keyword_name.value_counts().reset_index()

In [77]:
tag

,index,keyword_name
0,시리즈,32644
1,엔터,30040
2,사회정서,26135
3,언어1,23611
4,TV 방영,16553
...,...,...
1057,한부모가족,1
1058,태교이론,1
1059,보상방법,1
1060,용감함 4,1


In [78]:
with_key5 = meta_plus_df.loc[meta_plus_df['keyword_value'] == 5].drop_duplicates('album_id')

In [79]:
with_key5['keyword_name'].nunique()

191

In [8]:
with_key5

,album_id,keyword_type,keyword_name,keyword_value
7,749,05020201,스토리 애니,5
28,750,05020201,스토리 애니,5
49,2131,05020201,스토리 애니,5
72,2625,05020201,스토리 애니,5
92,2594,05020201,스토리 애니,5
...,...,...,...,...
767903,39872,05030702,그림그리기/색칠하기,5
767910,39873,05030702,그림그리기/색칠하기,5
767917,39874,05030702,그림그리기/색칠하기,5
767928,4779,05020204,동화,5


In [80]:
with_key5_count = with_key5['keyword_name'].value_counts().reset_index()
with_key5_count.columns = ['keyword', 'count']
with_key5_count.loc[with_key5_count['count'] > 50]

,keyword,count
0,스토리 애니,12481
1,어린이 예능,7822
2,노래,4922
3,교육,2783
4,인지,1906
5,동화,1842
6,어린이 드라마,768
7,율동,656
8,유치원/어린이집과 친구,642
9,동식물과 자연,571


In [81]:
keyword_list = with_key5_count.loc[with_key5_count['count'] > 50].keyword.to_list()

In [106]:
print(keyword_list)

['스토리 애니', '어린이 예능', '노래', '교육', '인지', '동화', '어린이 드라마', '율동', '유치원/어린이집과 친구', '동식물과 자연', '공연', '놀이', '건강과 안전', '나와 가족', '교통기관', '장난감 놀이', '생활도구', '음악', '언어지능 1', '점토 놀이', '우리 동네', '로보카폴리', '봄, 여름, 가을, 겨울', '신체운동지능', '타요', '미술', '색깔', '특별한 날', '악기연주']


In [14]:
len(keyword_list)

29

In [83]:
cnt = 0
cnt_notinlist = 0
for i in with_key5.itertuples():
    cnt += 1
    if i[3] not in keyword_list:
        cnt_notinlist += 1

In [16]:
cnt_notinlist

1218

In [99]:
keyword_list2 = with_key5_count.loc[with_key5_count['count'] > 100].keyword.to_list()
cnt1 = 0
cnt_notinlist1 = 0
for i in with_key5.itertuples():
    cnt1 += 1
    if i[3] not in keyword_list2:
        cnt_notinlist1 += 1

In [100]:
cnt_notinlist1

1782

In [84]:
tag_df = meta_plus_df.loc[(meta_plus_df['keyword_value'] == 5)][['album_id', 'keyword_name']]

In [36]:
tag_df

,album_id,keyword_name
7,749,스토리 애니
11,749,타요
14,749,버스
28,750,스토리 애니
34,750,타요
...,...,...
767917,39874,그림그리기/색칠하기
767928,4779,동화
767929,4779,창작동화
767938,11629,노래


In [108]:
from collections import defaultdict
tag_dict = defaultdict(list)
tag_cnt = 0
for i in tag_df.itertuples():
    if i[2] in keyword_list and i[2] not in tag_dict[i[1]]:
        tag_dict[i[1]].append(i[2])
        tag_cnt += 1

In [109]:
tag_cnt

51291

In [87]:
tag_dict

defaultdict(list,
            {749: ['스토리 애니', '타요'],
             750: ['스토리 애니', '타요'],
             2131: ['스토리 애니', '타요'],
             2625: ['스토리 애니', '타요'],
             2594: ['스토리 애니', '타요'],
             2637: ['스토리 애니', '타요'],
             2636: ['스토리 애니', '타요'],
             748: ['스토리 애니', '타요'],
             1381: ['스토리 애니', '타요'],
             1380: ['스토리 애니', '타요'],
             746: ['스토리 애니', '타요'],
             745: ['스토리 애니', '타요'],
             744: ['스토리 애니', '타요'],
             628: ['스토리 애니', '타요'],
             627: ['스토리 애니', '타요'],
             626: ['스토리 애니', '타요'],
             631: ['스토리 애니', '타요'],
             630: ['스토리 애니', '타요'],
             629: ['스토리 애니', '타요'],
             6744: ['스토리 애니', '타요'],
             7037: ['스토리 애니', '타요'],
             668: ['스토리 애니', '타요'],
             632: ['스토리 애니', '타요'],
             817: ['스토리 애니', '타요'],
             816: ['스토리 애니', '타요'],
             815: ['스토리 애니', '타요'],
             26077: ['스토리 애니'],
     

In [88]:
MAX = 0
for i in tag_dict.values():
    MAX = max(MAX, len(i))

In [112]:
for key, i in tag_dict.items():
    if len(i) == 7:
        print(key, i)

486 ['인지', '나와 가족', '건강과 안전', '생활도구', '봄, 여름, 가을, 겨울', '언어지능 1', '신체운동지능']
11556 ['인지', '나와 가족', '우리 동네', '건강과 안전', '생활도구', '언어지능 1', '신체운동지능']
14649 ['유치원/어린이집과 친구', '동식물과 자연', '건강과 안전', '생활도구', '특별한 날', '언어지능 1', '신체운동지능']
2439 ['인지', '동식물과 자연', '건강과 안전', '색깔', '미술', '음악', '신체운동지능']
5280 ['인지', '동식물과 자연', '건강과 안전', '봄, 여름, 가을, 겨울', '특별한 날', '음악', '언어지능 1']
5282 ['인지', '우리 동네', '건강과 안전', '봄, 여름, 가을, 겨울', '특별한 날', '음악', '언어지능 1']
22048 ['인지', '우리 동네', '건강과 안전', '봄, 여름, 가을, 겨울', '특별한 날', '음악', '언어지능 1']


In [114]:
# 39875개 아이템 중 38052개 아이템은 유의미하고 자주 쓰이는 태그 있음
# 1823개는 그런 태그 없음
len(tag_dict.keys())

38052

In [115]:
meta_df['album_id'].nunique()

39875

In [118]:
tag_dict.values()

dict_values([['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니', '타요'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['스토리 애니'], ['나와 가족', '언어지능 1'], ['유치원/어린이집과 친구', '언어지능 1'], ['언어지능 1'], ['유치원/어린이집과 친구', '언어지능 1'], ['유치원/어린이집과 친구', '언어지능 1'], ['유치원/어린이집과 친구', '언어지능 1'], ['유치원/어린이집과 친구', '언어지능 1'], ['건강과 안전', '교통기관', '언어지능 1'], ['유치원/어린이집과 친구', '언어지능 1'], ['유치원/어린이집과 친구', '언어지능 1'], ['언어지능 1'], ['유치원/어린이집과 친구', '언어지능 1'], ['언어지능 1'], ['스토리 애니'], ['스토

In [116]:
from collections import defaultdict
tag_dict2 = defaultdict(list)
tag_cnt2 = 0
for i in tag_df.itertuples():
    if i[2] in keyword_list2 and i[2] not in tag_dict2[i[1]]:
        tag_dict2[i[1]].append(i[2])
        tag_cnt2 += 1

In [117]:
# 100으로 설정하면 약 2190개 행 처리 어려움
len(tag_dict2.keys())

37685

In [105]:
keyword_list2

['스토리 애니',
 '어린이 예능',
 '노래',
 '교육',
 '인지',
 '동화',
 '어린이 드라마',
 '율동',
 '유치원/어린이집과 친구',
 '동식물과 자연',
 '공연',
 '놀이',
 '건강과 안전',
 '나와 가족',
 '교통기관',
 '장난감 놀이',
 '생활도구',
 '음악',
 '언어지능 1',
 '점토 놀이']

In [119]:
cnt_multi_tag = 0
for key, i in tag_dict.items():
    if len(i) > 1:
        cnt_multi_tag += 1

cnt_multi_tag
# 39875개 아이템 중 두 개 이상의 태그를 갖는 아이템은 총 9637개

9637

In [127]:
cnt_two_tag = 0
for key, i in tag_dict.items():
    if len(i) == 2:
        cnt_two_tag += 1

cnt_two_tag
# 39875개 아이템 중 두 개의 태그를 갖는 아이템은 총 6771개

6771

In [165]:
# 장르랑 merge해볼까
tag_df = tag_df.reset_index()[['album_id', 'keyword_name']]

In [168]:
tag_main_key = tag_df.iloc[0:1, :]
cnt_concat = 0
for i in tag_df.itertuples():
    if i[2] in keyword_list:
        tag_main_key = pd.concat([tag_main_key, tag_df.iloc[i[0]:i[0] + 1, :]])
        cnt_concat += 1

tag_main_key = tag_main_key.iloc[1:, :]

In [182]:
tag_main_key

,album_id,keyword_name
0,749,스토리 애니
1,749,타요
3,750,스토리 애니
4,750,타요
6,2131,스토리 애니
...,...,...
104838,39868,어린이 예능
104839,39869,어린이 예능
104845,4779,동화
104847,11629,노래


In [170]:
tag_genre = pd.merge(tag_main_key, meta_df[['album_id', 'genre_mid']], left_on = 'album_id', right_on = 'album_id', how = 'outer').fillna('nan')
tag_genre

,album_id,keyword_name,genre_mid
0,749,스토리 애니,TV만화
1,749,스토리 애니,TV만화
2,749,타요,TV만화
3,750,스토리 애니,TV만화
4,750,타요,TV만화
...,...,...,...
57106,39870,nan,놀이교실
57107,39871,nan,놀이교실
57108,39872,nan,놀이교실
57109,39873,nan,놀이교실


# 최빈키워드처럼 최빈 태그만 남기기

In [184]:
new_we = we_df[['profile_id', 'album_id']].drop_duplicates(['profile_id', 'album_id'])

In [185]:
new_we

,profile_id,album_id
0,3,15
1,3,16
2,3,17
3,3,18
4,3,19
...,...,...
892777,33032,0
892778,33032,1725
892780,33032,3128
892782,33032,3625


In [190]:
new_k = pd.merge(new_we, tag_main_key, left_on = 'album_id', right_on = 'album_id', how = 'inner')

In [194]:
new_k

,profile_id,album_id,keyword_name
0,3,15,율동
1,5,15,율동
2,22,15,율동
3,96,15,율동
4,101,15,율동
...,...,...,...
441767,32965,25913,노래
441768,32965,17289,노래
441769,32965,25914,노래
441770,32965,25915,노래


In [197]:
count_tag = tag_main_key.groupby('keyword_name')['album_id'].count()

In [208]:
count_tag

keyword_name
건강과 안전             753
공연                 486
교육                2865
교통기관               434
나와 가족              435
노래                4973
놀이                3917
동식물과 자연           1113
동화                1843
로보카폴리              506
미술                 182
봄, 여름, 가을, 겨울      159
색깔                 488
생활도구               354
스토리 애니           12481
신체운동지능             662
악기연주                53
어린이 드라마            768
어린이 예능            8295
언어지능 1            2199
우리 동네              126
유치원/어린이집과 친구       764
율동                 982
음악                2286
인지                1920
장난감 놀이            1256
점토 놀이              135
타요                 807
특별한 날              255
Name: album_id, dtype: int64

In [199]:
new_k2 = pd.DataFrame(new_k.groupby(['album_id','keyword_name']).count()).reset_index()

In [201]:
new_k2

,album_id,keyword_name,profile_id
0,0,"봄, 여름, 가을, 겨울",787
1,0,언어지능 1,787
2,1,건강과 안전,142
3,1,신체운동지능,142
4,1,음악,142
...,...,...,...
28258,25912,동화,1
28259,25913,노래,1
28260,25914,노래,1
28261,25915,노래,1


In [205]:
tag_cnt = new_k2.pivot(index = 'album_id', columns = 'keyword_name', values = 'profile_id').reset_index().fillna(0)

In [209]:
for i in range(29):
    tag_cnt.iloc[:,i+1] = tag_cnt.iloc[:, i + 1] / count_tag[i]

In [212]:
tag_cnt.columns

Index(['album_id', '건강과 안전', '공연', '교육', '교통기관', '나와 가족', '노래', '놀이',
       '동식물과 자연', '동화', '로보카폴리', '미술', '봄, 여름, 가을, 겨울', '색깔', '생활도구', '스토리 애니',
       '신체운동지능', '악기연주', '어린이 드라마', '어린이 예능', '언어지능 1', '우리 동네',
       '유치원/어린이집과 친구', '율동', '음악', '인지', '장난감 놀이', '점토 놀이', '타요', '특별한 날'],
      dtype='object', name='keyword_name')

In [214]:
tag_melt = tag_cnt.melt(id_vars = "album_id",
                        value_vars = ['건강과 안전', '공연', '교육', '교통기관', '나와 가족', '노래', '놀이',
                                      '동식물과 자연', '동화', '로보카폴리', '미술', '봄, 여름, 가을, 겨울',
                                      '색깔', '생활도구', '스토리 애니', '신체운동지능', '악기연주',
                                      '어린이 드라마', '어린이 예능', '언어지능 1', '우리 동네',
                                      '유치원/어린이집과 친구', '율동', '음악', '인지', '장난감 놀이',
                                      '점토 놀이', '타요', '특별한 날'],
                        value_name = 'viewpercell',
                        var_name = "keyword_name"
                       )

In [216]:
tag_max = tag_melt.loc[tag_melt.groupby(['album_id'])['viewpercell'].idxmax()]

In [217]:
tag_max

,album_id,keyword_name,viewpercell
212421,0,"봄, 여름, 가을, 겨울",4.949686
289666,1,신체운동지능,0.214502
212423,2,"봄, 여름, 가을, 겨울",0.779874
57936,3,교통기관,0.276498
270358,4,스토리 애니,0.010496
...,...,...,...
173794,25912,동화,0.000543
115862,25913,노래,0.000201
115863,25914,노래,0.000201
115864,25915,노래,0.000201


In [244]:
pure_album = meta_df.drop_duplicates('album_id')[['album_id', 'genre_mid', 'sub_title']]

In [245]:
pure_album

,album_id,genre_mid,sub_title
0,749,TV만화,꼬마버스 타요1
1,750,TV만화,꼬마버스 타요1
2,2131,TV만화,꼬마버스 타요1
3,2625,TV만화,꼬마버스 타요1
4,2594,TV만화,꼬마버스 타요1
...,...,...,...
42596,39872,놀이교실,로티프렌즈 미술놀이
42597,39873,놀이교실,로티프렌즈 미술놀이
42598,39874,놀이교실,로티프렌즈 미술놀이
42599,4779,책,4-5세


In [250]:
tag_genre_subtitle = pd.merge(pure_album, tag_max[['album_id', 'keyword_name']], on = 'album_id', how = 'left')

In [251]:
tag_genre_subtitle

,album_id,genre_mid,sub_title,keyword_name
0,749,TV만화,꼬마버스 타요1,타요
1,750,TV만화,꼬마버스 타요1,타요
2,2131,TV만화,꼬마버스 타요1,타요
3,2625,TV만화,꼬마버스 타요1,타요
4,2594,TV만화,꼬마버스 타요1,타요
...,...,...,...,...
39870,39872,놀이교실,로티프렌즈 미술놀이,NaN
39871,39873,놀이교실,로티프렌즈 미술놀이,NaN
39872,39874,놀이교실,로티프렌즈 미술놀이,NaN
39873,4779,책,4-5세,NaN


In [256]:
tag_and_sub = tag_genre[['sub_title', 'keyword_name']].value_counts().reset_index().sort_values('sub_title')

In [257]:
tag_and_sub

,sub_title,keyword_name,0
2751,100분! 뽀요 인기 메들리,노래,1
2640,10월 세계 여러나라,교육,1
2113,11월 지구와 우주,교육,1
2089,12월 겨울,교육,1
2063,1월 생활도구,어린이 예능,1
...,...,...,...
1471,활짝(5단계) 레벨,인지,3
1324,활짝(5단계) 레벨,유치원/어린이집과 친구,4
290,활짝(5단계) 레벨,건강과 안전,16
2021,흘려듣기,인지,1


In [258]:
subtitle_tag_dict = dict()

temp = 'a'
MAX = 0
for i in tag_and_sub.itertuples():
    if temp == i[1]:
        MAX = max(MAX, i[3])
        if MAX == i[3]:
            subtitle_tag_dict[i[1]] = i[2]
    else:
        temp = i[1]
        subtitle_tag_dict[i[1]] = i[2]

In [259]:
subtitle_tag_dict

{'100분! 뽀요 인기 메들리': '노래',
 '10월 세계 여러나라': '교육',
 '11월 지구와 우주': '교육',
 '12월 겨울': '교육',
 '1월 생활도구': '어린이 예능',
 '2월 자연과 동물': '교육',
 '3세이하': '노래',
 '3월 새로움이 많아요': '교육',
 '4-5세': '동화',
 '4남매쇼': '어린이 예능',
 '4월 봄': '교육',
 '5가지 뽀롱뽀롱 꿈 상담소': '어린이 예능',
 '5분 종이접기 한반도의 공룡편': '놀이',
 '5월 몸짱 마음짱': '교육',
 '6-7세': '동화',
 '60분 뽀요 율동 메들리': '율동',
 '6월 우리동네': '교육',
 '7월 여름과 건강': '교육',
 '8-9세': '동화',
 '8월 교통기관': '교육',
 '9월 우리나라': '교육',
 'A Day with Little Monkey': '생활도구',
 'A New Baby Is Coming': '나와 가족',
 'A Picnic Day': '동식물과 자연',
 'A Super Halloween': '특별한 날',
 'A Wish For Whales': '음악',
 'ABC Song': '인지',
 'Alligator Eats Candies': '미술',
 'Animals at the Zoo': '신체운동지능',
 'Baby Bear`s New Friend': '봄, 여름, 가을, 겨울',
 'Bath Time Fun': '나와 가족',
 'Ben Bakes Cakes': '언어지능 1',
 'Ben Is Too Big': '음악',
 'Big Big Big Bread': '언어지능 1',
 'Big Bob Small Sam': '건강과 안전',
 'Brown Bear`s Birthday': '나와 가족',
 'Cat and Rat Play Together': '음악',
 'Collin Goes to a Toilet': '건강과 안전',
 'Decorate a Chrismas Tree': '특별한 날',
 '

In [260]:
tag_genre_sub_copy = tag_genre_subtitle.fillna('nan')
tag_genre_sub_copy

,album_id,genre_mid,sub_title,keyword_name
0,749,TV만화,꼬마버스 타요1,타요
1,750,TV만화,꼬마버스 타요1,타요
2,2131,TV만화,꼬마버스 타요1,타요
3,2625,TV만화,꼬마버스 타요1,타요
4,2594,TV만화,꼬마버스 타요1,타요
...,...,...,...,...
39870,39872,놀이교실,로티프렌즈 미술놀이,nan
39871,39873,놀이교실,로티프렌즈 미술놀이,nan
39872,39874,놀이교실,로티프렌즈 미술놀이,nan
39873,4779,책,4-5세,nan


In [261]:
# 39875개 아이템 중 20564개가 결측
tag_genre_sub_copy.loc[tag_genre_sub_copy['keyword_name'] == 'nan']

,album_id,genre_mid,sub_title,keyword_name
26,26077,애니,변신자동차 또봇2,nan
27,26078,애니,변신자동차 또봇2,nan
28,26079,애니,변신자동차 또봇2,nan
30,26080,애니,변신자동차 또봇1,nan
31,13771,애니,변신자동차 또봇1,nan
...,...,...,...,...
39870,39872,놀이교실,로티프렌즈 미술놀이,nan
39871,39873,놀이교실,로티프렌즈 미술놀이,nan
39872,39874,놀이교실,로티프렌즈 미술놀이,nan
39873,4779,책,4-5세,nan


In [265]:
need_replace_cnt = 0

for i in tag_genre_sub_copy.itertuples():
    if i[4] == 'nan':
        if i[3] in subtitle_tag_dict.keys():
            tag_genre_sub_copy.loc[i[0], 'keyword_name'] = subtitle_tag_dict[i[3]]
        else:
            need_replace_cnt += 1

In [266]:
# 20564개 중에서 5571개를 제외한 같은 subtitle의 태그로 대체됨
need_replace_cnt

5571

In [268]:
# 39875개 아이템 중 20564개가 결측
tag_genre_sub_copy.loc[tag_genre_sub_copy['keyword_name'] == 'nan']

,album_id,genre_mid,sub_title,keyword_name
182,26106,놀이교실,숲 속 친구 파파룰라1,nan
183,26107,놀이교실,숲 속 친구 파파룰라1,nan
184,26108,놀이교실,숲 속 친구 파파룰라1,nan
185,26109,놀이교실,숲 속 친구 파파룰라1,nan
186,26110,예능,리얼리티쇼 유아독존,nan
...,...,...,...,...
39869,39871,놀이교실,로티프렌즈 미술놀이,nan
39870,39872,놀이교실,로티프렌즈 미술놀이,nan
39871,39873,놀이교실,로티프렌즈 미술놀이,nan
39872,39874,놀이교실,로티프렌즈 미술놀이,nan


In [293]:
genre_tag_cnt_df = tag_genre_sub_copy[['genre_mid', 'keyword_name']].value_counts().reset_index()
genre_tag_cnt_df.columns = ['genre_mid', 'keyword_name', 'count']
genre_tag_dict = genre_tag_cnt_df.loc[genre_tag_cnt_df.groupby('genre_mid')['count'].idxmax()][['genre_mid', 'keyword_name']].set_index('genre_mid').to_dict()['keyword_name']
genre_tag_dict

{'MCN': '어린이 예능',
 'TV만화': '스토리 애니',
 '극장판 애니': '타요',
 '노래 율동': '노래',
 '노래율동': '노래',
 '놀이교실': '어린이 예능',
 '다큐멘터리': 'nan',
 '독서동화': '동화',
 '드라마': 'nan',
 '수학과학': '교육',
 '시리즈': '스토리 애니',
 '시사교양': '공연',
 '애니': '스토리 애니',
 '액션/모험': '스토리 애니',
 '어린이방송': '스토리 애니',
 '역사/문화': 'nan',
 '연예오락': 'nan',
 '예능': '어린이 예능',
 '예술': 'nan',
 '외국어': '인지',
 '유아애니': '스토리 애니',
 '육아정보': 'nan',
 '창의학습': '놀이',
 '책': '동화',
 '체육안전': '교육',
 '코미디': '스토리 애니',
 '한글': '교육',
 '한자': '스토리 애니',
 '휴먼/감동': 'nan'}

In [296]:
need_replace = 0
execption = 0

for i in tag_genre_sub_copy.itertuples():
    
    if i[4] == 'nan':
        
        need_replace += 1
        
        if i[2] in genre_tag_dict.keys():
            tag_genre_sub_copy.loc[i[0], 'keyword_name'] = genre_tag_dict[i[2]]
        else:
            tag_genre_sub_copy.loc[i[0], 'keyword_name'] = '교육'
            print(i)
            execption += 1

In [300]:
tag_genre_sub_copy.loc[tag_genre_sub_copy['keyword_name'] == 'nan']

,album_id,genre_mid,sub_title,keyword_name


In [301]:
tag_genre_sub_copy

,album_id,genre_mid,sub_title,keyword_name
0,749,TV만화,꼬마버스 타요1,타요
1,750,TV만화,꼬마버스 타요1,타요
2,2131,TV만화,꼬마버스 타요1,타요
3,2625,TV만화,꼬마버스 타요1,타요
4,2594,TV만화,꼬마버스 타요1,타요
...,...,...,...,...
39870,39872,놀이교실,로티프렌즈 미술놀이,교육
39871,39873,놀이교실,로티프렌즈 미술놀이,교육
39872,39874,놀이교실,로티프렌즈 미술놀이,교육
39873,4779,책,4-5세,동화


In [298]:
need_replace

0

In [297]:
execption

0

In [304]:
mode_tag_for_each_album = tag_genre_sub_copy[['album_id', 'keyword_name']]
mode_tag_for_each_album.columns = ['album_id', 'tag']

In [307]:
mode_tag_for_each_album

,album_id,tag
0,749,타요
1,750,타요
2,2131,타요
3,2625,타요
4,2594,타요
...,...,...
39870,39872,교육
39871,39873,교육
39872,39874,교육
39873,4779,동화


In [306]:
mode_tag_for_each_album.tag.value_counts()

스토리 애니           10264
교육                8140
노래                3782
어린이 예능            3717
놀이                2652
동화                1813
장난감 놀이            1035
율동                 869
인지                 763
타요                 743
동식물과 자연            587
건강과 안전             492
공연                 485
어린이 드라마            477
신체운동지능             469
로보카폴리              464
교통기관               398
특별한 날              382
유치원/어린이집과 친구       382
음악                 330
언어지능 1             291
생활도구               277
나와 가족              274
색깔                 214
봄, 여름, 가을, 겨울      182
미술                 132
점토 놀이              108
우리 동네              100
악기연주                53
Name: tag, dtype: int64

# to pickle

In [308]:
with open('tag_for_all_id.pickle', 'wb') as f:
    pickle.dump(mode_tag_for_each_album, f)